# Preliminaries

In [7]:
import csv
import os
#from pyspark import SparkContext
#import numpy as np
#sc = SparkContext("local", "encoding program")

# Key generation

We start our RSA algorithm by choosing some parameters. First, we need to select two primes:

In [8]:
p=65497
q=66361

We proceed by calculating their product $n$, and the number of divisors of $n$, denoted normally by $\phi (n)$

In [9]:
n=p*q
phi_n=(p-1)*(q-1)

Next, we choose a number which is coprime to $n$. This should theoretically be random, but to make our calculations easier later, we choose $e$ to be $65537=2^{16}+1$ which is prime, and hence coprime to $n$.

In [10]:
e=65537

This algorithm is to calculate the inverse of $e$ modulo $\phi(n)$.

In [11]:
def euclidAlg(x,y):
    gcd=1
    if y>x:
        u=x
        x=y
        y=u
    fst=x
    scnd=y
    while x%y!=0:
        gcd=x%y
        x=y
        y=gcd
    a_list=[]
    y_list=[0]
    x1=fst
    x2=scnd
    yi=0
    while yi!=gcd:
        yi=x1%x2
        ai=(x1-yi)/x2
        a_list.append(ai)
        y_list.append(yi)
        x1=x2
        x2=yi
    c1=1
    c2=-1*a_list[-1]
    for i in range(1,(len(a_list))):
        if i%2 == 1:
            c1=c1+(-1*c2)*a_list[-(i+1)]
        else:
            c2=c2+(-1*c1)*a_list[-(i+1)]
    if len(a_list)%2==0:
        m1=scnd
        m2=fst
        d=c1
    else:
        m1=fst
        m2=scnd
        d=c2
    print("%i * %i + %i * %i = %i" % (c1, m1, c2, m2, gcd))
    return(d)

In [12]:
d=euclidAlg(phi_n,e)

4405 * 4346314560 + -292132927 * 65537 = 1


Since $\phi(n)=4346314560$, taking both sides modulo $\phi(n)$ yeilds that the inverse of $e\;(65537)$ is $-292132927$, or $4054181633$. This extra step is necessary for real RSA, but will not affect us here since we will not try to decode.

In [22]:
pub_key=[e,n]
priv_key=[d,n]

# Encoding

First we need the binary representation of the document we are encrypting, in a list

In [23]:
path=os.getcwd()
fileread=open(path+"/DoiBin.csv", "r")
data=[int(x) for x in list(csv.reader(fileread))[0]]

Now we parallelise the list to enable it to work in pySpark. We also need a second copy of the original data, which we need for the final calculation $ m^e \;=\; m^{{2^16}+1}\;=\;m^{{2^16}} \times m$, so to encrypt $m$ we simply need to square it sixteen times and multiply by $m$, reducing modulo $n$

In [24]:
rdd=sc.parallelize(data)
md=rdd.map(lambda a: a)

We now perform the calculation described above. Having parallelised the data, it is simple to write functions to square each entry and reduce modulo $n$, sixteen times

In [25]:
for f in range(1,17):
    squ=md.map(lambda a: a**2)
    md=squ.map(lambda a: a%n)

We now perform the final step, multiplying by the original value of $m$, and formatting back into a list

In [26]:
a=np.array(rdd.collect())
b=np.array(md.collect())
C=list(a*b)
E=[x%n for x in C]

These are the first few values in our encrypted list. Without knowing the prime decomposition of $n$, an attacker cannot decrypt (since they need to know $d$, which requires knowing $\phi(n)$, which requires knowing the values of $p$ and $q$). So cracking the code is as hard as recovering $p$ and $q$ from $n$, which is more difficult as $n$ gets larger.

In [31]:
E[0:10]

[57268, 6968, 27530, 17915, 40406, 47315, 13736, 52806, 35109, 47162]